In [ ]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import sys
import os
import numpy as np

class LSGAN():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='mse',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generated imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(z, valid)
        # (!!!) Optimize w.r.t. MSE loss instead of crossentropy
        self.combined.compile(loss='mse', optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        # (!!!) No softmax
        model.add(Dense(1))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise as generator input
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_imgs = self.generator.predict(noise)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, valid)
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch:%d step:%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, global_step,d_loss[0], 100 * d_loss[1], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)



    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_lsgan'):
            os.mkdir('images_lsgan')
        fig.savefig("images_lsgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    gan = LSGAN()
    gan.train(epochs=30, batch_size=64, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
_________________________________________________________________
____

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1 [D loss: 0.292869, acc.: 68.75%] [G loss: 0.837729]
epoch:0 step:2 [D loss: 2.628053, acc.: 39.84%] [G loss: 1.100568]
epoch:0 step:3 [D loss: 0.548781, acc.: 38.28%] [G loss: 0.949492]
epoch:0 step:4 [D loss: 0.190073, acc.: 78.91%] [G loss: 0.935954]
epoch:0 step:5 [D loss: 0.263412, acc.: 62.50%] [G loss: 0.866519]
epoch:0 step:6 [D loss: 0.198448, acc.: 75.00%] [G loss: 0.851447]
epoch:0 step:7 [D loss: 0.141225, acc.: 82.81%] [G loss: 1.020959]
epoch:0 step:8 [D loss: 0.133630, acc.: 81.25%] [G loss: 1.052579]
epoch:0 step:9 [D loss: 0.144996, acc.: 81.25%] [G loss: 0.980982]
epoch:0 step:10 [D loss: 0.122165, acc.: 85.94%] [G loss: 0.926761]
epoch:0 step:11 [D loss: 0.099432, acc.: 90.62%] [G loss: 1.027935]
epoch:0 step:12 [D loss: 0.163766, acc.: 76.56%] [G loss: 0.920839]
epoch:0 step:13 [D loss: 0.132623, acc.: 82.81%] [G loss: 1.037970]
epoch:0 step:14 [D loss: 0.218907, acc.: 67.97%] [G loss: 0.887577]
epoch:0 step:15 [D loss: 0.469538, acc.: 40.62%] [G loss:

epoch:0 step:123 [D loss: 0.039231, acc.: 97.66%] [G loss: 1.127414]
epoch:0 step:124 [D loss: 0.048097, acc.: 96.09%] [G loss: 0.833695]
epoch:0 step:125 [D loss: 0.044648, acc.: 96.09%] [G loss: 1.014460]
epoch:0 step:126 [D loss: 0.036737, acc.: 98.44%] [G loss: 0.970165]
epoch:0 step:127 [D loss: 0.039463, acc.: 98.44%] [G loss: 1.161327]
epoch:0 step:128 [D loss: 0.051928, acc.: 96.88%] [G loss: 0.945276]
epoch:0 step:129 [D loss: 0.068303, acc.: 91.41%] [G loss: 0.842481]
epoch:0 step:130 [D loss: 0.082397, acc.: 89.06%] [G loss: 0.971017]
epoch:0 step:131 [D loss: 0.102991, acc.: 83.59%] [G loss: 0.956188]
epoch:0 step:132 [D loss: 0.127116, acc.: 82.81%] [G loss: 1.020632]
epoch:0 step:133 [D loss: 0.162610, acc.: 79.69%] [G loss: 0.924660]
epoch:0 step:134 [D loss: 0.192542, acc.: 74.22%] [G loss: 0.976635]
epoch:0 step:135 [D loss: 0.185557, acc.: 71.88%] [G loss: 0.901361]
epoch:0 step:136 [D loss: 0.087966, acc.: 90.62%] [G loss: 1.051427]
epoch:0 step:137 [D loss: 0.081878

...

epoch:29 step:27449 [D loss: 0.185195, acc.: 75.00%] [G loss: 0.434749]
epoch:29 step:27450 [D loss: 0.228340, acc.: 61.72%] [G loss: 0.443784]
epoch:29 step:27451 [D loss: 0.238569, acc.: 60.16%] [G loss: 0.455585]
epoch:29 step:27452 [D loss: 0.217923, acc.: 63.28%] [G loss: 0.415463]
epoch:29 step:27453 [D loss: 0.210998, acc.: 58.59%] [G loss: 0.450604]
epoch:29 step:27454 [D loss: 0.244201, acc.: 57.81%] [G loss: 0.431790]
epoch:29 step:27455 [D loss: 0.220387, acc.: 64.84%] [G loss: 0.416377]
epoch:29 step:27456 [D loss: 0.191997, acc.: 73.44%] [G loss: 0.445258]
epoch:29 step:27457 [D loss: 0.205455, acc.: 70.31%] [G loss: 0.458502]
epoch:29 step:27458 [D loss: 0.239670, acc.: 59.38%] [G loss: 0.431670]
epoch:29 step:27459 [D loss: 0.196470, acc.: 71.09%] [G loss: 0.460756]
epoch:29 step:27460 [D loss: 0.257278, acc.: 54.69%] [G loss: 0.423632]
epoch:29 step:27461 [D loss: 0.194476, acc.: 72.66%] [G loss: 0.477083]
epoch:29 step:27462 [D loss: 0.184999, acc.: 72.66%] [G loss: 0.

epoch:29 step:27567 [D loss: 0.225856, acc.: 60.94%] [G loss: 0.413601]
epoch:29 step:27568 [D loss: 0.220078, acc.: 64.06%] [G loss: 0.412569]
epoch:29 step:27569 [D loss: 0.270988, acc.: 56.25%] [G loss: 0.441403]
epoch:29 step:27570 [D loss: 0.225336, acc.: 60.16%] [G loss: 0.478034]
epoch:29 step:27571 [D loss: 0.193721, acc.: 72.66%] [G loss: 0.481294]
epoch:29 step:27572 [D loss: 0.210274, acc.: 67.19%] [G loss: 0.451597]
epoch:29 step:27573 [D loss: 0.238781, acc.: 63.28%] [G loss: 0.384041]
epoch:29 step:27574 [D loss: 0.226528, acc.: 57.03%] [G loss: 0.433788]
epoch:29 step:27575 [D loss: 0.193947, acc.: 70.31%] [G loss: 0.431301]
epoch:29 step:27576 [D loss: 0.237504, acc.: 53.91%] [G loss: 0.369981]
epoch:29 step:27577 [D loss: 0.228868, acc.: 60.16%] [G loss: 0.433146]
epoch:29 step:27578 [D loss: 0.200917, acc.: 67.97%] [G loss: 0.420129]
epoch:29 step:27579 [D loss: 0.189060, acc.: 74.22%] [G loss: 0.480499]
epoch:29 step:27580 [D loss: 0.233725, acc.: 59.38%] [G loss: 0.

epoch:29 step:27681 [D loss: 0.198882, acc.: 67.97%] [G loss: 0.526363]
epoch:29 step:27682 [D loss: 0.257712, acc.: 63.28%] [G loss: 0.440745]
epoch:29 step:27683 [D loss: 0.250345, acc.: 55.47%] [G loss: 0.391406]
epoch:29 step:27684 [D loss: 0.239046, acc.: 53.12%] [G loss: 0.407963]
epoch:29 step:27685 [D loss: 0.217502, acc.: 67.19%] [G loss: 0.427089]
epoch:29 step:27686 [D loss: 0.190794, acc.: 66.41%] [G loss: 0.411239]
epoch:29 step:27687 [D loss: 0.212869, acc.: 64.84%] [G loss: 0.414352]
epoch:29 step:27688 [D loss: 0.203093, acc.: 64.84%] [G loss: 0.421447]
epoch:29 step:27689 [D loss: 0.204578, acc.: 71.88%] [G loss: 0.473117]
epoch:29 step:27690 [D loss: 0.262829, acc.: 51.56%] [G loss: 0.412332]
epoch:29 step:27691 [D loss: 0.224899, acc.: 63.28%] [G loss: 0.444625]
epoch:29 step:27692 [D loss: 0.198309, acc.: 67.97%] [G loss: 0.427838]
epoch:29 step:27693 [D loss: 0.213409, acc.: 67.19%] [G loss: 0.466761]
epoch:29 step:27694 [D loss: 0.201085, acc.: 67.97%] [G loss: 0.

epoch:29 step:27800 [D loss: 0.225724, acc.: 64.06%] [G loss: 0.386138]
epoch:29 step:27801 [D loss: 0.216780, acc.: 60.16%] [G loss: 0.418995]
epoch:29 step:27802 [D loss: 0.222789, acc.: 64.06%] [G loss: 0.412802]
epoch:29 step:27803 [D loss: 0.209332, acc.: 63.28%] [G loss: 0.434282]
epoch:29 step:27804 [D loss: 0.194831, acc.: 68.75%] [G loss: 0.446023]
epoch:29 step:27805 [D loss: 0.213437, acc.: 68.75%] [G loss: 0.422001]
epoch:29 step:27806 [D loss: 0.225049, acc.: 62.50%] [G loss: 0.462868]
epoch:29 step:27807 [D loss: 0.204655, acc.: 65.62%] [G loss: 0.451167]
epoch:29 step:27808 [D loss: 0.204770, acc.: 69.53%] [G loss: 0.466650]
epoch:29 step:27809 [D loss: 0.238359, acc.: 56.25%] [G loss: 0.446737]
epoch:29 step:27810 [D loss: 0.214573, acc.: 66.41%] [G loss: 0.442533]
epoch:29 step:27811 [D loss: 0.217327, acc.: 64.06%] [G loss: 0.430250]
epoch:29 step:27812 [D loss: 0.219210, acc.: 60.16%] [G loss: 0.453955]
epoch:29 step:27813 [D loss: 0.224287, acc.: 63.28%] [G loss: 0.

epoch:29 step:27920 [D loss: 0.212941, acc.: 65.62%] [G loss: 0.417272]
epoch:29 step:27921 [D loss: 0.249986, acc.: 56.25%] [G loss: 0.420413]
epoch:29 step:27922 [D loss: 0.238108, acc.: 53.91%] [G loss: 0.423784]
epoch:29 step:27923 [D loss: 0.216042, acc.: 61.72%] [G loss: 0.409296]
epoch:29 step:27924 [D loss: 0.210422, acc.: 67.97%] [G loss: 0.437758]
epoch:29 step:27925 [D loss: 0.226534, acc.: 66.41%] [G loss: 0.432364]
epoch:29 step:27926 [D loss: 0.231532, acc.: 58.59%] [G loss: 0.401594]
epoch:29 step:27927 [D loss: 0.199287, acc.: 71.09%] [G loss: 0.431642]
epoch:29 step:27928 [D loss: 0.213453, acc.: 68.75%] [G loss: 0.454759]
epoch:29 step:27929 [D loss: 0.212669, acc.: 65.62%] [G loss: 0.418581]
epoch:29 step:27930 [D loss: 0.254659, acc.: 59.38%] [G loss: 0.440506]
epoch:29 step:27931 [D loss: 0.252250, acc.: 59.38%] [G loss: 0.400354]
epoch:29 step:27932 [D loss: 0.230150, acc.: 60.16%] [G loss: 0.423415]
epoch:29 step:27933 [D loss: 0.233840, acc.: 62.50%] [G loss: 0.